In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
import numpy as np
from keras.models import Model

In [18]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#x_train = x_train.reshape(60000, 784)
#x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


60000 train samples
10000 test samples


In [20]:
#inputs = Input(shape=(28,28,1))
model = []
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1),padding='valid', activation='relu',input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=1, padding="valid", data_format=None))
model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(1, 1),strides=(1, 1),padding='valid',activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024,  activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, use_bias=False, activation='linear', name='svm'))
#x = model_1()
#x_out  = tf.keras.layers.Dense(10, use_bias=False, activation='linear', name='svm')(x)
#model = Model(inputs, x)

In [21]:
def svm_loss(layer):
    weights = layer.weights[0]
    weights_tf = tf.convert_to_tensor(weights)

    def categorical_hinge_loss(y_true, y_pred):
        pos = K.sum(y_true * y_pred, axis=-1)
        neg = K.max((1.0 - y_true) * y_pred, axis=-1)
        hinge_loss = K.mean(K.maximum(0.0, neg - pos + 1), axis=-1)
        regularization_loss = 0.5*(tf.reduce_sum(tf.square(weights_tf)))
        return regularization_loss + 0.4*hinge_loss

    return categorical_hinge_loss


In [22]:
metrics = ['accuracy']
#optimizer = tf.keras.optimizers.RMSprop(lr=2e-3, decay=1e-5)
optimizer = tf.keras.optimizers.Adam(1.e-3)

model.compile(optimizer=optimizer, loss=svm_loss(model.get_layer('svm')), metrics=metrics)

In [23]:
batch_size = 64
epochs = 4

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/4
938/938 [==============================] - 22s 13ms/step - loss: 10.0337 - accuracy: 0.9466 - val_loss: 9.9947 - val_accuracy: 0.9853
Epoch 2/4
938/938 [==============================] - 12s 13ms/step - loss: 9.9949 - accuracy: 0.9805 - val_loss: 9.9903 - val_accuracy: 0.9863
Epoch 3/4
938/938 [==============================] - 12s 12ms/step - loss: 9.9904 - accuracy: 0.9844 - val_loss: 9.9883 - val_accuracy: 0.9861
Epoch 4/4
938/938 [==============================] - 12s 12ms/step - loss: 9.9872 - accuracy: 0.9880 - val_loss: 9.9871 - val_accuracy: 0.9876
Test loss: 9.987115859985352
Test accuracy: 0.9876000285148621


In [ ]:
!nvidia-smi

Tue Apr 19 14:24:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    27W /  70W |    278MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------